In [1]:
import numpy as np
from functools import reduce
from itertools import combinations
from scipy.special import binom
import random
import time

m=8
r=4

In [2]:
def message_length(m, r):
    return sum([binom(m,i) for i in range(r+1)])

In [3]:
def generator_matrix(m, r):
    # associate x_i to the bitset with of length 2^m
    # of the form 2^(m-i-1) 1's followed by 2^(m-i-1) 0's
    # and so on, call this bitset nu(x_i)
    single_var_monomials=[np.array((2**(m-i-1)*[1]+2**(m-i-1)*[0])*2**i) for i in range(m)]
    
    # associate x_i_1*...*x_i_k to
    # nu(x_i_1)*...*nu(x_i_k) (bitwise multiplication)
    mult_var_monomials=[reduce(lambda v,w:v*w, [single_var_monomials[i] for i in factors]) 
                        for num_factors in range(2,r+1)
                        for factors in combinations(range(m), num_factors)]
    
    # finally, associate 1 to the bitset of 2^m 1's, 
    # put all the matrices together and compose the 
    # resulting matrix (for easier multiplication later))
    if r==1:
        return np.transpose(np.concatenate((np.array([[1]*2**m]), single_var_monomials)))
    else:
        return np.transpose(np.concatenate((np.array([[1]*2**m]), single_var_monomials, mult_var_monomials)))

def all_complementary_polynomials(compl_factors, m):
    # generate all "polynomials" of degree |compl_factors| consisting
    # of x_i's and (1-x_i)'s' with i in compl_factors
    
    if len(compl_factors)==0:
        return [np.array([1]*2**m)]
    
    x_i=compl_factors[0]
    compl_factors_rest=compl_factors[1:]
    
    x_i_row=np.array(([1]*2**(m-x_i-1)+[0]*2**(m-x_i-1))*2**x_i)
    x_i_row_inv=np.array([1]*2**m)-x_i_row
    
    compl_polys_rest=all_complementary_polynomials(compl_factors_rest,m)
    
    return [x_i_row*poly for poly in compl_polys_rest] + [x_i_row_inv*poly for poly in compl_polys_rest]

def all_S_fs(m,r):
    # find S_f for all f in M_m
    return [all_complementary_polynomials([i for i in range(m) if i not in factors], m)
            for num_factors in range(r+1)
            for factors in combinations(range(m), num_factors)]

def shorten_S_fs(m,r,S):
    req_size=2**(m-r)
    for j in range(len(S)):
        #print(len(S[j]))
        S[j]=S[j][:req_size]
    return S
def encode_message(message, M):
    return np.array([np.sum(message*col)%2 for col in M])

def decode_codeword(codeword, M, S, m, r): #TODO only 2^(m-r) estimates
    message=[]
    k=Integer(message_length(m,r))
    for j in range(k-1, -1, -1):
        vote=[0,0]
        for s in S[j]:
            vote[np.sum(codeword*s)%2]+=1
        if vote[0]==vote[1]:
            message+=[-1]
        elif vote[0]>vote[1]:
            message.insert(0,0)
        else:
            message.insert(0,1)
            codeword=(codeword-np.transpose(M)[j])%2
            
    return np.array(message)

In [10]:
M=generator_matrix(m,r)
S=all_S_fs(m,r)
S=shorten_S_fs(m,r,S)
print(len(S[0]))
k=Integer(message_length(m,r))
count=0
for blub in range(5):
    message=np.array([0]*k)
    for i in random.sample(range(k), k-10):
            message[i]=1
    codeword=encode_message(message,M)
    
    for bla in range(2):
        error=np.array([0]*(2**m))
        for i in random.sample(range(2**m), 2**(m-r-1)-4):
            error[i]=1
        received=(codeword+error)%2
        tic=time.perf_counter()
        decoded_message=decode_codeword(received,M,S,m,r)
        toc=time.perf_counter()
        print(toc-tic)
        if not (decoded_message==message).all():
            print("Fail")

16
0.11213750002207235
0.11912069999380037
0.185853999981191
0.12636059999931604
0.11787289998028427
0.1250869000214152
0.1401171999750659
0.14607799996156245
0.12955059995874763
0.12257290002889931


In [314]:
def foo(x):
    

df=np.array([0,1,2,3,4])
print(df[:3])


[0 1 2]


In [169]:
print(np.array(range(5,-1,-1)))

[5 4 3 2 1 0]


In [184]:
M=np.array([np.array([1,2,3]),
           np.array([4,5,6])])
np.transpose(M)[1]

array([2, 5])

In [67]:
a = np.array([[1, 2], [3, 4]])

b = np.array([[5, 6]])

c = np.array([[7, 8]])

np.concatenate((a, b, c), axis=0)

array([[1, 2],
       [3, 4],
       [5, 6],
       [7, 8]])

In [304]:
import time
print(time.perf_counter())
for i in range(10000000):
    a=3*4
print(time.perf_counter())

210507.315969
210511.0631707


In [234]:
print(type(np.zeros(5)))

<class 'numpy.ndarray'>
